# 이미지 특징 추출

### 이미지 미분
가로/세로 위치 변화에 따른 픽셀 값의 변화율을 이미지의 도함수(Image derivatives) 라고 한다. 다음 식에서 𝑓(𝑥)는 𝑥위치의 픽셀 명도이다.

$ g_{x} = \frac{∂𝑓}{∂𝑥}, g_{y} = \frac{∂𝑓}{∂𝑦} $  

x, y 방향의 도함수의 크기를 구하는 것이 라플라스 연산이다.  

$ g = \sqrt{{g_{x}}^{2} + {g_{y}}^{2}} $  

실제 이미지 프로세싱에서는 다음과 같은 중앙차분법(central difference)을 사용한다.  

$ G(x) = f(x+1) - f(x-1) \approx \begin{bmatrix} -1 & 0 & 1 \end{bmatrix} $  

이 연산은 다음 이미지 커널을 사용하여 필터링한 것과 같다.  
$ k = [1, 0, -1] $